In [ ]:
# Install compatible TensorFlow and tensorflow-privacy(sol)
# !pip uninstall tensorflow -y
# !pip uninstall tensorflow-privacy -y 
# !pip install tensorflow==2.14.0
# !pip install tensorflow-privacy==0.8.0

In [ ]:
!pip install deap

In [ ]:
!pip install --upgrade tensorflow-estimator==2.3.0

In [ ]:
# =========================
# Standard Library
# =========================
import os
import random
import time
import warnings

# =========================
# Third-Party: Numerical / Data
# =========================
import numpy as np
import pandas as pd

# =========================
# Third-Party: Visualization
# =========================
import matplotlib.pyplot as plt
import seaborn as sns

# =========================
# Third-Party: sklearn
# =========================
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.utils import resample, shuffle
from sklearn.metrics import (
    accuracy_score,
    average_precision_score,
    auc,
    confusion_matrix,
    f1_score,
    precision_recall_curve,
    precision_score,
    recall_score,
)

# =========================
# Third-Party: Imbalanced-Learn
# =========================
from imblearn.over_sampling import RandomOverSampler

# =========================
# Third-Party: TensorFlow & Privacy
# =========================
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    LSTM,
    Dense,
    Dropout,
    Flatten,
    Conv2D,
    MaxPooling2D,
)
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import (
    TruePositives,
    TrueNegatives,
    FalsePositives,
    FalseNegatives,
)


# !pip install tensorflow-privacy==0.8.0
import tensorflow_privacy

from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

# =========================
# Third-Party: Genetic Algorithm (DEAP)
# =========================
from deap import base, creator, tools

# =========================
# Warnings
# =========================
warnings.filterwarnings("ignore", category=UserWarning)



## Load and Preprocess Data

In [ ]:
data  = pd.read_csv(r"C:\Users\dmachooka\Downloads\dmac\IoT_Intrusion.csv")


In [ ]:
# Multiclass problem
target_column = 'label'
display(data[target_column].unique())
num_classes = data[target_column].nunique()

In [ ]:
data[target_column].value_counts() # Imbalance learning not feasible, simplify the problem lets only predict for top 3 conditions

In [ ]:
# simplify the problem lets only predict for top 3 conditions
data = data[data[target_column].isin(['DDoS-ICMP_Flood','DDoS-UDP_Flood','DDoS-TCP_Flood'])]

In [ ]:
# Label encode the target column
from sklearn.preprocessing import LabelEncoder

# Instantiate LabelEncoder
label_encoder = LabelEncoder()

# Encode the target column
data[target_column] = label_encoder.fit_transform(data[target_column])

In [ ]:
# Identify non-numeric columns (there are none)
print(data.select_dtypes(include='object').columns)

In [ ]:
categorical_columns = [col for col in data.columns if col not in [target_column]] # Other columns

X = data.drop(target_column, axis=1)
y = data[target_column]

# Split into training test and validation datasets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42)

X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.4, random_state=42)



# Standard Scaling
scaler = StandardScaler()

#Perfom feature scaling for the training set
X_train_scaled = X_train.copy()
X_train_scaled[categorical_columns] = scaler.fit_transform(X_train[categorical_columns])

#Perfom feature scaling for the training set
X_valid_scaled = X_valid.copy()
X_valid_scaled[categorical_columns] = scaler.transform(X_valid[categorical_columns])

#Perfom feature scaling for the test set
X_test_scaled = X_test.copy()
X_test_scaled[categorical_columns] = scaler.transform(X_test[categorical_columns])

In [ ]:
# Function to plot loss curves
def plot_loss(history1, history2=None):
    plt.figure(figsize=(12, 4))

    # Plot training & validation loss values for the first model
    plt.subplot(1, 2, 1)
    plt.plot(history1.history['loss'])
    plt.plot(history1.history['val_loss'])
    plt.title('Model loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper right')

    # Plot training & validation accuracy values for the first model
    plt.subplot(1, 2, 2)
    plt.plot(history1.history['accuracy'])
    plt.plot(history1.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='lower right')

    if history2 is not None:
        # Plot training & validation loss values for the second model
        plt.subplot(1, 2, 1)
        plt.plot(history2.history['loss'], linestyle='dashed')
        plt.plot(history2.history['val_loss'], linestyle='dashed')
        plt.legend(['Train', 'Validation', 'Train Privacy', 'Validation Privacy'], loc='upper right')

        # Plot training & validation accuracy values for the second model
        plt.subplot(1, 2, 2)
        plt.plot(history2.history['accuracy'], linestyle='dashed')
        plt.plot(history2.history['val_accuracy'], linestyle='dashed')
        plt.legend(['Train', 'Validation', 'Train Privacy', 'Validation Privacy'], loc='lower right')

    plt.tight_layout()
    plt.show()

## GA Privacy

In [ ]:
# --- Define parameter search space ---
LSTM_UNITS = [16, 32, 64, 128, 256]
DENSE_UNITS = [16, 32, 64, 128, 256]
ACTIVATIONS = ['gelu', 'swish', 'silu', 'relu']
OPTIMIZERS = ['sgd', 'adam']
LOSSES = ['sparse_categorical_crossentropy']
LEARNING_RATES = [0.001, 0.0001]
BATCH_SIZES = [64, 128, 256]
EPOCHS = [20, 30, 50, 100]
MAX_LSTM_LAYERS = 3
MAX_DENSE_LAYERS = 4
noise_multiplier=1.3
gene_alg = True

In [ ]:
def evaluate_nn_dp(hyperparams):
    # Unpack hyperparameters
    lstm_units = hyperparams['lstm_units']
    dense_units = hyperparams['dense_units']
    activation = hyperparams['activation']
    optimizer_choice = hyperparams['optimizer']
    loss_choice = hyperparams['loss']
    learning_rate = hyperparams['learning_rate']
    batch_size = hyperparams['batch_size']
    epochs = hyperparams['epochs']
    dropouts = hyperparams['dropouts']
    l2_norm_clip = hyperparams['l2_norm_clip']
    noise_multiplier = hyperparams['noise_multiplier']

    # Define sequential model
    model = Sequential()

    # Input LSTM layers
    for i, units in enumerate(lstm_units):
        return_sequences = True if i < len(lstm_units) - 1 else False
        model.add(LSTM(units=units, activation=activation,
                       return_sequences=return_sequences,
                       input_shape=(1,len(categorical_columns))))
        model.add(Dropout(dropouts[i]))

    # Dense layers
    for j, units in enumerate(dense_units):
        model.add(Dense(units=units, activation=activation))
        # can remove len(lstm_units)+ make it equall
        model.add(Dropout(dropouts[len(lstm_units) + j])) 

    # Output layer
    model.add(Dense(num_classes, activation='softmax'))

    # DP Optimizer
    if optimizer_choice == "adam":
        base_optimizer = Adam(learning_rate=learning_rate)
    else:
        base_optimizer = SGD(learning_rate=learning_rate, momentum=0.9)

    dp_optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
        l2_norm_clip=l2_norm_clip,
        noise_multiplier=noise_multiplier,
        num_microbatches=1,
        learning_rate=learning_rate
    )

    model.compile(optimizer=dp_optimizer, loss=loss_choice, metrics=['accuracy'])

    # Perform Stratified K-Fold CV
    kfold = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
    cv_results = []

    for train_index, test_index in kfold.split(X_train_scaled[categorical_columns], y_train):
        X_train_fold, X_val_fold = (
            X_train_scaled[categorical_columns].iloc[train_index],
            X_train_scaled[categorical_columns].iloc[test_index]
        )
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[test_index]

        # Reshape for LSTM: (samples, timesteps=1, features)
        X_train_fold = np.expand_dims(X_train_fold, axis=1)
        X_val_fold = np.expand_dims(X_val_fold, axis=1)

        # Train model
        model.fit(X_train_fold, y_train_fold, epochs=epochs, batch_size=batch_size, verbose=0)
        # _, val_accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
        # cv_results.append(val_accuracy)
        # Predict class labels for F1
        y_pred_proba = model.predict(X_val_fold, verbose=0)
        y_pred = np.argmax(y_pred_proba, axis=1)   # convert softmax → class

        # Compute F1
        # f1 = f1_score(y_val_fold, y_pred, average="binary")  # or "macro", "weighted"
        f1 = f1_score(y_val_fold, y_pred, average="macro")
        cv_results.append(f1)

    # Extract trainable variables from the model
    trainable_variables = model.trainable_variables

    # Compute privacy budget
    privacy_report = compute_dp_sgd_privacy.compute_dp_sgd_privacy_statement(
        number_of_examples=len(X_train_scaled),
        batch_size=batch_size,
        noise_multiplier=1.3,
        num_epochs=epochs,
        delta=1e-5
    )

    epsilon = None

    # Case A: report is an object with epsilon attr
    if hasattr(privacy_report, "epsilon"):
        epsilon = privacy_report.epsilon

    # Case B: report is a tuple (epsilon, something)
    elif isinstance(privacy_report, tuple) and len(privacy_report) >= 1 and isinstance(privacy_report[0], (float, int)):
        epsilon = privacy_report[0]

    # Case C: report is a string
    elif isinstance(privacy_report, str):
        import re
        match = re.search(r"epsilon[^0-9]*([0-9.]+)", privacy_report.lower())
        if match:
            epsilon = float(match.group(1))

    score = np.mean(cv_results) - epsilon # GA maximizes score by default

    return score,

In [ ]:
def custom_crossover(ind1, ind2):
    keys = list(ind1.keys())
    cxpoint1, cxpoint2 = sorted([random.randint(1, len(keys)-1) for _ in range(2)])
    for i in range(cxpoint1, cxpoint2):
        key = keys[i]
        ind1[key], ind2[key] = ind2[key], ind1[key]
    return ind1, ind2

def custom_mutation(ind):
    param_to_mutate = random.choice(list(ind.keys()))

    if param_to_mutate == 'lstm_units':
        ind['lstm_units'] = [random.choice(LSTM_UNITS) for _ in range(len(ind['lstm_units']))]
    elif param_to_mutate == 'lstm_dropouts':
        ind['lstm_dropouts'] = [random.uniform(0.1, 0.5) for _ in range(len(ind['lstm_dropouts']))]
    elif param_to_mutate == 'dense_units':
        ind['dense_units'] = [random.choice(DENSE_UNITS) for _ in range(len(ind['dense_units']))]
    elif param_to_mutate == 'dense_dropouts':
        ind['dense_dropouts'] = [random.uniform(0.1, 0.5) for _ in range(len(ind['dense_dropouts']))]
    elif param_to_mutate == 'activation':
        ind['activation'] = random.choice(ACTIVATIONS)
    elif param_to_mutate == 'optimizer':
        ind['optimizer'] = random.choice(OPTIMIZERS)
    elif param_to_mutate == 'loss':
        ind['loss'] = random.choice(LOSSES)
    elif param_to_mutate == 'learning_rate':
        ind['learning_rate'] = random.choice(LEARNING_RATES)
    elif param_to_mutate == 'batch_size':
        ind['batch_size'] = random.choice(BATCH_SIZES)
    elif param_to_mutate == 'epochs':
        ind['epochs'] = random.choice(EPOCHS)

    return ind,

In [ ]:
def create_hyperparameter_set_dp():
    # Random number of LSTM and Dense layers
    n_lstm = random.randint(1, MAX_LSTM_LAYERS)
    n_dense = random.randint(1, MAX_DENSE_LAYERS)

    return {
        'lstm_units': [random.choice(LSTM_UNITS) for _ in range(n_lstm)],
        'dense_units': [random.choice(DENSE_UNITS) for _ in range(n_dense)],
        'activation': random.choice(ACTIVATIONS),
        'optimizer': random.choice(OPTIMIZERS),
        'loss': random.choice(LOSSES),
        'learning_rate': random.choice(LEARNING_RATES),
        'batch_size': random.choice(BATCH_SIZES),
        'epochs': random.choice(EPOCHS),
        'dropouts': [random.uniform(0.1, 0.5) for _ in range(n_lstm + n_dense)],
        'l2_norm_clip': random.uniform(0.1, 5.0),
        # 'noise_multiplier': random.uniform(0.5, 2.0) has been hard coded 1.3 at top
        'noise_multiplier': random.uniform(1.3, 1.3)
    }


# === DEAP Setup ===
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", dict, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, create_hyperparameter_set_dp)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# --- NOTE: custom crossover must be defined ---
toolbox.register("mate", custom_crossover)

# Mutate: random re-sample of new hyperparams
def custom_mutate(individual):
    new_params = create_hyperparameter_set_dp()
    individual.update(new_params)
    return individual,

toolbox.register("mutate", custom_mutate)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate_nn_dp)

In [ ]:
def genetic_algorithm_dp():
    pop = toolbox.population(n=10)
    ngen = 5  # Number of generations
    cxpb, mutpb = 0.5, 0.2  # Crossover and mutation probabilities

    for gen in range(ngen):
        print(f"-- Generation {gen} --")

        # Evaluate all individuals
        fitnesses = list(map(toolbox.evaluate, pop))
        for ind, fit in zip(pop, fitnesses):
            ind.fitness.values = fit

        # Select individuals for the next generation
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < cxpb:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < mutpb:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        pop[:] = offspring

    # Return the best individual
    top_individual = tools.selBest(pop, 1)[0]
    return top_individual

In [ ]:
# debuging
# dir(best_hyperparameters_dp = genetic_algorithm_dp())

In [ ]:
# Run the genetic algorithm
gene_alg = True
if gene_alg:
  best_hyperparameters_dp = genetic_algorithm_dp()
  print("Best Hyperparameters with DP:", best_hyperparameters_dp)

In [ ]:
if gene_alg:
    best_hp_dp = {
        'lstm_units': best_hyperparameters_dp['lstm_units'],
        'dense_units': best_hyperparameters_dp['dense_units'],
        'activation': best_hyperparameters_dp['activation'],
        'optimizer': best_hyperparameters_dp['optimizer'],
        'loss': best_hyperparameters_dp['loss'],
        'learning_rate': best_hyperparameters_dp['learning_rate'],
        'batch_size': best_hyperparameters_dp['batch_size'],
        'epochs': best_hyperparameters_dp['epochs'],
        'dropouts': best_hyperparameters_dp['dropouts'],
        'l2_norm_clip': best_hyperparameters_dp['l2_norm_clip'],
        'noise_multiplier': best_hyperparameters_dp['noise_multiplier']
    }
else:
    best_hp_dp = {
        'lstm_units': [64, 128],   # Example manual config
        'dense_units': [256, 128], # Example manual config
        'activation': 'relu',
        'optimizer': 'sgd',
        'loss': 'sparse_categorical_crossentropy',
        'learning_rate': 0.001,
        'batch_size': 64,
        'epochs': 30,
        'dropouts': [0.2, 0.3, 0.25, 0.2],  # one per layer (LSTM + Dense)
        'l2_norm_clip': 1.0,
        'noise_multiplier': 1.3
    }

# --- Build the final LSTM+Dense DP model ---
model_privacy = tf.keras.Sequential()

# Add LSTM layers
for i, units in enumerate(best_hp_dp['lstm_units']):
    return_sequences = (i < len(best_hp_dp['lstm_units']) - 1)
    model_privacy.add(tf.keras.layers.LSTM(
        units=units,
        activation=best_hp_dp['activation'],
        return_sequences=return_sequences,
        input_shape=(1,len(categorical_columns))
    ))
    model_privacy.add(Dropout(rate=best_hp_dp['dropouts'][i]))

# Add Dense layers
for j, units in enumerate(best_hp_dp['dense_units']):
    model_privacy.add(Dense(units=units, activation=best_hp_dp['activation']))
    model_privacy.add(Dropout(rate=best_hp_dp['dropouts'][len(best_hp_dp['lstm_units']) + j]))

# Output layer
model_privacy.add(Dense(num_classes, activation='softmax'))

# DP optimizer
dp_optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=best_hp_dp['l2_norm_clip'],
    noise_multiplier=best_hp_dp['noise_multiplier'],
    num_microbatches=1,
    learning_rate=best_hp_dp['learning_rate']
)

# Compile model
model_privacy.compile(optimizer=dp_optimizer, loss=best_hp_dp['loss'], metrics=['accuracy'])

# --- Cross-validation ---
kfold = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
cv_results = []

for train_index, test_index in kfold.split(X_train_scaled[categorical_columns], y_train):
    X_train_fold, X_val_fold = (
        X_train_scaled[categorical_columns].iloc[train_index].values.reshape(-1,1, len(categorical_columns)),
        X_train_scaled[categorical_columns].iloc[test_index].values.reshape(-1,1, len(categorical_columns)),
    )
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    print("hi")
    model_privacy.fit(
        X_train_fold, y_train_fold,
        epochs=best_hp_dp['epochs'],
        batch_size=best_hp_dp['batch_size'],
        verbose=0
    )
    val_loss, val_accuracy = model_privacy.evaluate(X_val_fold, y_val_fold, verbose=0)
    cv_results.append([val_loss, val_accuracy])

# # --- Final training on full data ---
X_train_lstm = X_train_scaled[categorical_columns].values.reshape(-1,1, len(categorical_columns))
X_test_lstm = X_test_scaled[categorical_columns].values.reshape(-1,1, len (categorical_columns))

# Reshape test data for LSTM;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
# X_test_lstm = X_test_scaled[categorical_columns].values.reshape(-1, 1, len(categorical_columns))

# Verify shapes
print("X_test_scaled[categorical_columns] shape:", X_test_scaled[categorical_columns].values.shape)  # (38378, 46)
print("X_test_lstm shape:", X_test_lstm.shape)  # Should be (38378, 1, 46)
print("Model input shape:", model_privacy.input_shape)  # (None, 1, 46)

# Predict on the reshaped test data
y_test_pred_mlp_privacy = model_privacy.predict(X_test_lstm, verbose=0)

# Apply threshold for binary predictions
y_test_pred_binary_mlp_privacy = y_test_pred_mlp_privacy.argmax(axis=1)  # For softmax/multi-class
# OR for binary classification with sigmoid:
# y_test_pred_binary_mlp_privacy = (y_test_pred_mlp_privacy > 0.5).astype(int)
# ';;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;



model_history_privacy = model_privacy.fit(
    X_train_lstm, y_train,
    epochs=best_hp_dp['epochs'],
    batch_size=best_hp_dp['batch_size'],
    validation_split=0.2
)

# --- Evaluate on test set ---
test_loss, test_accuracy = model_privacy.evaluate(X_test_lstm, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


In [ ]:
# confirming reshape;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
print(X_test_scaled[categorical_columns].values.shape)
print(len(categorical_columns))
print(model_privacy.input_shape)
# ;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;

In [ ]:
# Predict on the test data
y_test_pred_mlp_privacy = model_privacy.predict(X_test_scaled[categorical_columns].values.reshape(-1,1, len (categorical_columns)))

# # Apply the threshold to make binary predictions
y_test_pred_binary_mlp_privacy = y_test_pred_mlp_privacy.argmax(axis=1)

In [ ]:
# Print the average cross-validation results
print("Average cross-validation loss:", sum(cv_result[0] for cv_result in cv_results) / len(cv_results))
print("Average cross-validation accuracy:", sum(cv_result[1] for cv_result in cv_results) / len(cv_results))

# Evaluate the MLP model
accuracy_mlp_privacy = accuracy_score(y_test, y_test_pred_binary_mlp_privacy)
precision_mlp_privacy = precision_score(y_test, y_test_pred_binary_mlp_privacy, average='macro')
recall_mlp_privacy = recall_score(y_test, y_test_pred_binary_mlp_privacy, average='macro')
f1_mlp_privacy = f1_score(y_test, y_test_pred_binary_mlp_privacy, average='macro')

# Print the results for the MLP model
print("MLP Model:")
print(f"Accuracy =  {accuracy_mlp_privacy}")
print(f"Precision = {precision_mlp_privacy}")
print(f"Recall = {recall_mlp_privacy}")
print(f"F1 Score =  {f1_mlp_privacy}")

In [ ]:
# Plot loss curves
plot_loss(model_history_privacy)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_test_pred_binary_mlp_privacy)

plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()

num_classes = conf_matrix.shape[0]

# Get class labels from the LabelEncoder
class_labels = label_encoder.classes_

plt.xticks(np.arange(num_classes), class_labels, rotation=45)
plt.yticks(np.arange(num_classes), class_labels)

thresh = conf_matrix.max() / 2.

# Add text annotations
for i in range(num_classes):
    for j in range(num_classes):
        plt.text(j, i, format(conf_matrix[i, j], 'd'),
                 ha="center", va="center",
                 color="white" if conf_matrix[i, j] > thresh else "black")

plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.tight_layout()
plt.show()

# Precision-Recall Curve
# precision, recall, thresholds = precision_recall_curve(y_test_binary_true, y_test_pred_binary_mlp_privacy)
# area_under_curve = auc(recall, precision)

# plt.figure(figsize=(8, 6))
# plt.plot(recall, precision, label=f'Privacy Model Precision-Recall Curve (AUC = {area_under_curve:.2f})')
# plt.xlabel('Recall')
# plt.ylabel('Precision')
# plt.title('Privacy Model Precision-Recall Curve')
# plt.legend(loc='lower left')
# plt.show()

In [ ]:
# Extract trainable variables from the model
trainable_variables = model_privacy.trainable_variables

#correct function
privacy_report = compute_dp_sgd_privacy.compute_dp_sgd_privacy_statement(
    number_of_examples=len(X_train_scaled),
    batch_size=32,
    noise_multiplier=1.099235020314155,
    num_epochs=30,
    delta=1e-5
)

# Compute privacy budget
#privacy_report = compute_dp_sgd_privacy.compute_dp_sgd_privacy_statement(
   # number_of_examples=len(X_train_scaled),
   # batch_size=batch_size,
   # noise_multiplier=noise_multiplier,
   # num_epochs=3,
   # delta=1e-5
#)

In [ ]:
print(privacy_report)

## Combine Results

In [ ]:
# # Create DataFrame
# data = {
#     'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
#     'Regular': [accuracy_lstm, precision_lstm, recall_lstm, f1_lstm],
#     'Privacy': [accuracy_mlp_privacy, precision_mlp_privacy, recall_mlp_privacy, f1_mlp_privacy]
# }

# df = pd.DataFrame(data)

# # Print DataFrame
# display(df)

In [ ]:
# plot_loss(history, model_history_privacy)